## YOLO Fine-tuning on Pascal VOC Dataset
Complete pipeline for training, image detection, and video detection

[Pascal VOC Dataset source](https://www.kaggle.com/datasets/gopalbhattrai/pascal-voc-2012-dataset)

In [1]:
%%capture
!pip install ultralytics

In [2]:
import os
import shutil
from pathlib import Path
import cv2
import torch
from ultralytics import YOLO
import yaml
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# ============================================================================
# KAGGLE DATASET PATHS
# ============================================================================

KAGGLE_VOC_PATH = '/kaggle/input/pascal-voc-2012-dataset'
KAGGLE_TRAIN_VAL_PATH = '/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val'
KAGGLE_TEST_PATH = '/kaggle/input/pascal-voc-2012-dataset/VOC2012_test'

# Working directory (writable in Kaggle)
WORK_DIR = '/kaggle/working'
OUTPUT_DIR = os.path.join(WORK_DIR, 'voc_yolo_dataset')

In [ ]:
# ============================================================================
# DIAGNOSTIC FUNCTION - Run this first to check dataset
# ============================================================================

def diagnose_dataset():
    """Diagnose and find the Pascal VOC dataset location"""
    
    print("\n" + "="*70)
    print("DATASET DIAGNOSTICS")
    print("="*70 + "\n")
    
    print("Checking base path...")
    if os.path.exists(KAGGLE_VOC_PATH):
        print(f"✓ Base path exists: {KAGGLE_VOC_PATH}")
        print("\nContents:")
        for item in os.listdir(KAGGLE_VOC_PATH):
            print(f"  - {item}")
    else:
        print(f"✗ Base path not found: {KAGGLE_VOC_PATH}")
        return
    
    print("\n" + "-"*70)
    print("Searching for JPEGImages and Annotations folders...")
    print("-"*70 + "\n")
    
    found_datasets = []
    
    for root, dirs, files in os.walk(KAGGLE_VOC_PATH):
        if 'JPEGImages' in dirs and 'Annotations' in dirs:
            found_datasets.append(root)
            print(f"✓ Found dataset at: {root}")
            
            # Check contents
            jpeg_path = os.path.join(root, 'JPEGImages')
            ann_path = os.path.join(root, 'Annotations')
            
            jpeg_count = len([f for f in os.listdir(jpeg_path) if f.endswith('.jpg')])
            xml_count = len([f for f in os.listdir(ann_path) if f.endswith('.xml')])
            
            print(f"  Images: {jpeg_count}")
            print(f"  Annotations: {xml_count}")
            
            # Check for ImageSets
            imagesets = os.path.join(root, 'ImageSets', 'Main')
            if os.path.exists(imagesets):
                print(f"  ImageSets: ✓")
                if os.path.exists(os.path.join(imagesets, 'train.txt')):
                    print(f"    train.txt: ✓")
                if os.path.exists(os.path.join(imagesets, 'val.txt')):
                    print(f"    val.txt: ✓")
            print()
    
    if not found_datasets:
        print("✗ No Pascal VOC dataset structure found!")
        print("\nPlease verify:")
        print("1. Dataset is properly added to Kaggle notebook")
        print("2. Dataset name is correct: 'pascal-voc-2012-dataset'")
        print("3. Dataset is mounted at: /kaggle/input/")
    else:
        print(f"\n✓ Found {len(found_datasets)} dataset location(s)")
        print("\nRecommended path to use:")
        print(f"  {found_datasets[0]}")
    
    return found_datasets

In [ ]:
# ============================================================================
# STEP 1: Convert Pascal VOC XML to YOLO Format
# ============================================================================

# Pascal VOC class names
VOC_CLASSES = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person',
    'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

def convert_voc_to_yolo(xml_file, img_width, img_height):
    """
    Convert Pascal VOC XML annotation to YOLO format
    
    Returns: List of YOLO format annotations [class_id, x_center, y_center, width, height]
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    yolo_annotations = []
    
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        
        if class_name not in VOC_CLASSES:
            continue
            
        class_id = VOC_CLASSES.index(class_name)
        
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        # Convert to YOLO format (normalized x_center, y_center, width, height)
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        yolo_annotations.append([class_id, x_center, y_center, width, height])
    
    return yolo_annotations

def prepare_voc_dataset_for_yolo():
    """
    Prepare Pascal VOC dataset in YOLO format from Kaggle structure
    """
    
    print("\n" + "="*70)
    print("PREPARING PASCAL VOC DATASET FOR YOLO")
    print("="*70 + "\n")
    
    # Create output directory structure
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)
    
    # Process train/val split
    print("Processing train/val data...")
    print(f"Looking for dataset at: {KAGGLE_TRAIN_VAL_PATH}")
    
    # Try multiple possible paths
    possible_paths = [
        os.path.join(KAGGLE_TRAIN_VAL_PATH, 'VOCdevkit', 'VOC2012'),
        os.path.join(KAGGLE_TRAIN_VAL_PATH, 'VOC2012_train_val', 'VOCdevkit', 'VOC2012'),
        os.path.join(KAGGLE_TRAIN_VAL_PATH, 'VOC2012'),
        KAGGLE_TRAIN_VAL_PATH
    ]
    
    voc_root = None
    for path in possible_paths:
        if os.path.exists(os.path.join(path, 'JPEGImages')):
            voc_root = path
            print(f"✓ Found VOC dataset at: {voc_root}")
            break
    
    if voc_root is None:
        print(f"ERROR: Cannot find VOC dataset with JPEGImages folder")
        print("\nSearching for dataset structure...")
        for root, dirs, files in os.walk(KAGGLE_TRAIN_VAL_PATH):
            if 'JPEGImages' in dirs or 'Annotations' in dirs:
                print(f"  Found potential dataset at: {root}")
                voc_root = root
                break
        
        if voc_root is None:
            print("\nERROR: Could not locate Pascal VOC dataset structure!")
            print("Please check the dataset is properly mounted.")
            print(f"\nAvailable in {KAGGLE_TRAIN_VAL_PATH}:")
            try:
                for item in os.listdir(KAGGLE_TRAIN_VAL_PATH):
                    print(f"  - {item}")
            except:
                print("  Cannot list directory contents")
            return None
    
    # Read train/val split files
    imagesets_path = os.path.join(voc_root, 'ImageSets', 'Main')
    
    # Check if split files exist
    train_file = os.path.join(imagesets_path, 'train.txt')
    val_file = os.path.join(imagesets_path, 'val.txt')
    
    if not os.path.exists(train_file):
        # If split files don't exist, create them
        print("Split files not found. Creating train/val split (80/20)...")
        images_dir = os.path.join(voc_root, 'JPEGImages')
        all_images = [f.replace('.jpg', '') for f in os.listdir(images_dir) if f.endswith('.jpg')]
        
        np.random.shuffle(all_images)
        split_idx = int(len(all_images) * 0.8)
        train_ids = all_images[:split_idx]
        val_ids = all_images[split_idx:]
    else:
        with open(train_file, 'r') as f:
            train_ids = [line.strip() for line in f.readlines()]
        with open(val_file, 'r') as f:
            val_ids = [line.strip() for line in f.readlines()]
    
    print(f"Train images: {len(train_ids)}")
    print(f"Val images: {len(val_ids)}")
    
    # Process train set
    print("\nProcessing training set...")
    process_split(voc_root, train_ids, 'train')
    
    # Process val set
    print("Processing validation set...")
    process_split(voc_root, val_ids, 'val')
    
    # Process test set (if available)
    test_root = None
    test_possible_paths = [
        os.path.join(KAGGLE_TEST_PATH, 'VOCdevkit', 'VOC2012'),
        os.path.join(KAGGLE_TEST_PATH, 'VOC2012_test', 'VOCdevkit', 'VOC2012'),
        os.path.join(KAGGLE_TEST_PATH, 'VOC2012'),
        KAGGLE_TEST_PATH
    ]
    
    for path in test_possible_paths:
        if os.path.exists(os.path.join(path, 'JPEGImages')):
            test_root = path
            break
    
    if test_root and os.path.exists(os.path.join(test_root, 'JPEGImages')):
        print("\nProcessing test set...")
        test_images_dir = os.path.join(test_root, 'JPEGImages')
        test_images = [f.replace('.jpg', '') for f in os.listdir(test_images_dir) if f.endswith('.jpg')]
        process_split(test_root, test_images, 'test')
        print(f"Test images: {len(test_images)}")
    else:
        print("\nTest set not found - using only train/val splits")
    
    print("\n✓ Dataset preparation completed!")
    print(f"Dataset saved to: {OUTPUT_DIR}")
    
    return OUTPUT_DIR

def process_split(voc_root, image_ids, split_name):
    """Process a single split (train/val/test)"""
    
    images_path = os.path.join(voc_root, 'JPEGImages')
    annotations_path = os.path.join(voc_root, 'Annotations')
    
    output_images = os.path.join(OUTPUT_DIR, 'images', split_name)
    output_labels = os.path.join(OUTPUT_DIR, 'labels', split_name)
    
    processed = 0
    skipped = 0
    
    for img_id in tqdm(image_ids, desc=f"Processing {split_name}"):
        # Image file
        img_file = os.path.join(images_path, f"{img_id}.jpg")
        xml_file = os.path.join(annotations_path, f"{img_id}.xml")
        
        if not os.path.exists(img_file):
            skipped += 1
            continue
        
        # Read image to get dimensions
        img = cv2.imread(img_file)
        if img is None:
            skipped += 1
            continue
            
        img_height, img_width = img.shape[:2]
        
        # Convert annotations
        if os.path.exists(xml_file):
            yolo_annotations = convert_voc_to_yolo(xml_file, img_width, img_height)
        else:
            yolo_annotations = []
        
        # Skip images without annotations for train/val
        if len(yolo_annotations) == 0 and split_name in ['train', 'val']:
            skipped += 1
            continue
        
        # Copy image
        dst_img = os.path.join(output_images, f"{img_id}.jpg")
        shutil.copy(img_file, dst_img)
        
        # Write YOLO label file
        label_file = os.path.join(output_labels, f"{img_id}.txt")
        with open(label_file, 'w') as f:
            for ann in yolo_annotations:
                f.write(f"{ann[0]} {ann[1]:.6f} {ann[2]:.6f} {ann[3]:.6f} {ann[4]:.6f}\n")
        
        processed += 1
    
    print(f"  Processed: {processed}, Skipped: {skipped}")

In [ ]:
# ============================================================================
# STEP 2: Create YOLO Dataset Configuration
# ============================================================================

def create_voc_yaml(dataset_path):
    """Create YOLO dataset configuration file"""
    
    yaml_content = {
        'path': dataset_path,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        
        'names': {i: name for i, name in enumerate(VOC_CLASSES)},
        'nc': len(VOC_CLASSES)
    }
    
    yaml_path = os.path.join(WORK_DIR, 'voc.yaml')
    
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)
    
    print(f"\n✓ Created dataset config at: {yaml_path}")
    return yaml_path

In [ ]:
# ============================================================================
# STEP 3: Train YOLO Model
# ============================================================================

def train_yolo_voc_kaggle(model_size='n', epochs=100, batch_size=16, img_size=640):
    """
    Fine-tune YOLO on Pascal VOC for Kaggle environment
    
    Args:
        model_size: 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (xlarge)
        epochs: Number of training epochs
        batch_size: Batch size (adjust based on Kaggle GPU - usually 16 or 32)
        img_size: Input image size
    """
    
    print(f"\n{'='*70}")
    print(f"TRAINING YOLOv8{model_size.upper()} ON PASCAL VOC")
    print(f"{'='*70}\n")
    
    # Step 1: Prepare dataset
    dataset_path = prepare_voc_dataset_for_yolo()
    if dataset_path is None:
        print("ERROR: Dataset preparation failed!")
        return None
    
    # Step 2: Create config
    yaml_path = create_voc_yaml(dataset_path)
    
    # Step 3: Load pretrained model
    print(f"\nLoading YOLOv8{model_size} pretrained model...")
    model = YOLO(f'yolov8{model_size}.pt')
    
    # Check device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    if device == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Step 4: Train
    print(f"\nStarting training for {epochs} epochs...")
    print("This may take 2-4 hours depending on GPU...")
    
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        device=device,
        
        # Optimization
        optimizer='SGD',
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        
        # Augmentation
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=0.0,
        translate=0.1,
        scale=0.5,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.0,
        
        # Validation
        val=True,
        plots=True,
        save=True,
        save_period=20,
        
        # Paths
        project=os.path.join(WORK_DIR, 'runs/train'),
        name='yolo_voc',
        exist_ok=True,
        
        # Other
        patience=50,
        workers=4,  # Kaggle has limited CPU
        verbose=True,
        seed=42,
        
        # Close mosaic augmentation in final epochs
        close_mosaic=10
    )
    
    print("\n✓ Training completed!")
    best_model_path = os.path.join(WORK_DIR, 'runs/train/yolo_voc/weights/best.pt')
    print(f"Best model saved at: {best_model_path}")
    
    return model, best_model_path

In [ ]:
# ============================================================================
# STEP 4: Evaluate Model
# ============================================================================

def evaluate_model(model_path, yaml_path):
    """Evaluate trained model"""
    
    print("\n" + "="*70)
    print("EVALUATING MODEL")
    print("="*70 + "\n")
    
    model = YOLO(model_path)
    metrics = model.val(data=yaml_path)
    
    print(f"\n{'='*70}")
    print("EVALUATION METRICS:")
    print(f"{'='*70}")
    print(f"mAP50:       {metrics.box.map50:.4f}")
    print(f"mAP50-95:    {metrics.box.map:.4f}")
    print(f"Precision:   {metrics.box.mp:.4f}")
    print(f"Recall:      {metrics.box.mr:.4f}")
    print(f"{'='*70}\n")
    
    return metrics

In [ ]:
# ============================================================================
# STEP 5: Test Inference
# ============================================================================

def test_inference(model_path, test_images_dir=None, num_samples=5):
    """Test model on sample images"""
    
    print("\n" + "="*70)
    print("TESTING INFERENCE")
    print("="*70 + "\n")
    
    model = YOLO(model_path)
    
    # Get test images
    if test_images_dir is None:
        test_images_dir = os.path.join(OUTPUT_DIR, 'images', 'val')
    
    test_images = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir)[:num_samples]]
    
    print(f"Testing on {len(test_images)} sample images...\n")
    
    # Run inference
    results = model.predict(
        source=test_images,
        conf=0.25,
        iou=0.45,
        save=True,
        project=os.path.join(WORK_DIR, 'runs/detect'),
        name='test',
        exist_ok=True
    )
    
    # Print results
    for i, result in enumerate(results):
        print(f"\nImage {i+1}: {os.path.basename(result.path)}")
        print(f"Detections: {len(result.boxes)}")
        
        for box in result.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = result.names[cls]
            print(f"  - {class_name}: {conf:.2f}")
    
    save_path = os.path.join(WORK_DIR, 'runs/detect/test')
    print(f"\n✓ Results saved to: {save_path}")

In [4]:
# ============================================================================
# STEP 6: Export Model
# ============================================================================

def export_model_formats(model_path):
    """Export model to different formats"""
    
    print("\n" + "="*70)
    print("EXPORTING MODEL")
    print("="*70 + "\n")
    
    model = YOLO(model_path)
    
    # Export to ONNX (universal format)
    print("Exporting to ONNX...")
    onnx_path = model.export(format='onnx')
    print(f"✓ ONNX model: {onnx_path}")
    
    print("\n✓ Model export completed!")


In [5]:
# ============================================================================
# MAIN EXECUTION PIPELINE
# ============================================================================

def main():
    """Main training pipeline for Kaggle"""
    
    print("\n" + "="*70)
    print("YOLO PASCAL VOC FINE-TUNING - KAGGLE PIPELINE")
    print("="*70 + "\n")
    
    # First, diagnose the dataset
    print("Running dataset diagnostics...")
    found_datasets = diagnose_dataset()
    
    if not found_datasets:
        print("\n" + "="*70)
        print("CRITICAL ERROR: Dataset not found!")
        print("="*70)
        print("\nPlease:")
        print("1. Go to 'Add Data' in Kaggle notebook")
        print("2. Search for 'pascal voc 2012' or 'pascal-voc-2012-dataset'")
        print("3. Add the dataset to your notebook")
        print("4. Re-run this script")
        return
    
    # Configuration
    MODEL_SIZE = 'n'  # Options: 'n', 's', 'm', 'l', 'x'
    EPOCHS = 100      # Reduce to 50 for faster testing
    BATCH_SIZE = 16   # Adjust based on GPU memory (Kaggle P100: 16-32, T4: 8-16)
    IMG_SIZE = 640
    
    print("\n" + "="*70)
    print("TRAINING CONFIGURATION")
    print("="*70)
    print(f"  Model: YOLOv8{MODEL_SIZE}")
    print(f"  Epochs: {EPOCHS}")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Image size: {IMG_SIZE}")
    print()
    
    input("\nPress Enter to start training or Ctrl+C to abort...")
    
    try:
        # Step 1: Train model
        result = train_yolo_voc_kaggle(
            model_size=MODEL_SIZE,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            img_size=IMG_SIZE
        )
        
        if result is None:
            print("\n" + "="*70)
            print("ERROR: Training failed - dataset preparation issue")
            print("="*70)
            return
        
        model, best_model_path = result
        
        # Step 2: Evaluate
        yaml_path = os.path.join(WORK_DIR, 'voc.yaml')
        evaluate_model(best_model_path, yaml_path)
        
        # Step 3: Test inference
        test_inference(best_model_path, num_samples=5)
        
        # Step 4: Export model
        export_model_formats(best_model_path)
        
        print("\n" + "="*70)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*70)
        print(f"\nTrained model: {best_model_path}")
        print(f"Results directory: {WORK_DIR}/runs")
        print("\nTo download trained model, use:")
        print("  from IPython.display import FileLink")
        print(f"  FileLink('{best_model_path}')")
        
    except Exception as e:
        print("\n" + "="*70)
        print("ERROR: Pipeline failed")
        print("="*70)
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

In [6]:
if __name__ == "__main__":
    main()


YOLO PASCAL VOC FINE-TUNING - KAGGLE PIPELINE

Running dataset diagnostics...

DATASET DIAGNOSTICS

Checking base path...
✓ Base path exists: /kaggle/input/pascal-voc-2012-dataset

Contents:
  - VOC2012_train_val
  - VOC2012_test

----------------------------------------------------------------------
Searching for JPEGImages and Annotations folders...
----------------------------------------------------------------------

✓ Found dataset at: /kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val
  Images: 17125
  Annotations: 17125
  ImageSets: ✓
    train.txt: ✓
    val.txt: ✓

✓ Found dataset at: /kaggle/input/pascal-voc-2012-dataset/VOC2012_test/VOC2012_test
  Images: 16135
  Annotations: 5138
  ImageSets: ✓


✓ Found 2 dataset location(s)

Recommended path to use:
  /kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val

TRAINING CONFIGURATION
  Model: YOLOv8n
  Epochs: 100
  Batch size: 16
  Image size: 640




Press Enter to start training or Ctrl+C to abort... 



TRAINING YOLOv8N ON PASCAL VOC


PREPARING PASCAL VOC DATASET FOR YOLO

Processing train/val data...
Looking for dataset at: /kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val
ERROR: Cannot find VOC dataset with JPEGImages folder

Searching for dataset structure...
  Found potential dataset at: /kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val
Train images: 5717
Val images: 5823

Processing training set...


Processing train: 100%|██████████| 5717/5717 [01:58<00:00, 48.25it/s]


  Processed: 5717, Skipped: 0
Processing validation set...


Processing val: 100%|██████████| 5823/5823 [01:56<00:00, 49.87it/s]


  Processed: 5823, Skipped: 0

Test set not found - using only train/val splits

✓ Dataset preparation completed!
Dataset saved to: /kaggle/working/voc_yolo_dataset

✓ Created dataset config at: /kaggle/working/voc.yaml

Loading YOLOv8n pretrained model...
Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB

Starting training for 100 epochs...
This may take 2-4 hours depending on GPU...
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/voc.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, 

/usr/local/lib/python3.12/dist-packages/torch/onnx/utils.py:1397: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 5.4s, saved as '/kaggle/working/runs/train/yolo_voc/weights/best.onnx' (11.7 MB)

Export complete (5.7s)
Results saved to /kaggle/working/runs/train/yolo_voc/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/train/yolo_voc/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/train/yolo_voc/weights/best.onnx imgsz=640 data=/kaggle/working/voc.yaml  
Visualize:       https://netron.app
✓ ONNX model: /kaggle/working/runs/train/yolo_voc/weights/best.onnx

✓ Model export completed!

PIPELINE COMPLETED SUCCESSFULLY!

Trained model: /kaggle/working/runs/train/yolo_voc/weights/best.pt
Results directory: /kaggle/working/runs

To download trained model, use:
  from IPython.display import FileLink
  FileLink('/kaggle/working/runs/train/yolo_voc/weights/best.pt')
